In [ ]:
import pandas as pd
import numpy as np

CATEGORIES = [
    "Books",
    "Electronics",
    "Home_and_Kitchen",
    "Sports_and_Outdoors",
    "Toys_and_Games",
    "Beauty_and_Personal_Care",
    "Tools_and_Home_Improvement",
    "Grocery_and_Gourmet_Food",
    "Pet_Supplies",
    "Movies_and_TV",
]

TARGET_PER_CAT = 5000
BASE_URL = "https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/review_categories"

all_samples = []

for cat in CATEGORIES:
    print(f"\n🔹 Streaming category: {cat}")
    url = f"{BASE_URL}/{cat}.jsonl.gz"

    collected = []
    needed = TARGET_PER_CAT

    try:
        reader = pd.read_json(
            url,
            lines=True,
            compression="gzip",
            chunksize=50000,
        )

        for chunk in reader:
            # ✅ only keep columns that are guaranteed to exist
            cols_we_care = [c for c in ["rating", "text", "asin", "parent_asin"] if c in chunk.columns]
            chunk = chunk[cols_we_care].copy()
            chunk["category"] = cat

            # basic cleaning
            chunk = chunk.dropna(subset=["text", "rating"])
            chunk = chunk[chunk["text"].str.len() >= 30]
            chunk["rating"] = pd.to_numeric(chunk["rating"], errors="coerce")
            chunk = chunk[chunk["rating"].between(1, 5)]

            if chunk.empty:
                continue

            if needed <= 0:
                break

            # sample only what we still need
            if len(chunk) > needed:
                sample = chunk.sample(needed, random_state=42)
            else:
                sample = chunk

            collected.append(sample)
            needed -= len(sample)
            print(f"   Collected: {TARGET_PER_CAT - needed} reviews", end="\r")

            if needed <= 0:
                break

        if not collected:
            print(f"\n⚠️ No data collected for {cat}")
            continue

        cat_df = pd.concat(collected, ignore_index=True)
        print(f"\n✅ Final count for {cat}: {len(cat_df)}")
        all_samples.append(cat_df)

    except Exception as e:
        print(f"⚠️ Error reading {cat}: {e}")

# combine all categories
df = pd.concat(all_samples, ignore_index=True)
print("\n🎉 Total balanced dataset size:", len(df))
df.head()



🔹 Streaming category: Books
   Collected: 5000 reviews
✅ Final count for Books: 5000

🔹 Streaming category: Electronics
